<a href="https://colab.research.google.com/github/Akhaledelsayed/machine_learning_project-/blob/abdulla/my_svm_model__my_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Akhaledelsayed/machine_learning_project-.git

fatal: destination path 'machine_learning_project-' already exists and is not an empty directory.


In [2]:
import pandas as pd
import numpy as np
import cv2  # OpenCV for image loading
import os
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [3]:
IMAGE_FOLDER = "/content/machine_learning_project-/Project/Dataset/images"

# The CSV file containing the labels
CSV_PATH = "/content/Image_data.csv"

# Image size (smaller = faster training)
IMG_SIZE = (64, 64)

In [4]:
print(f"Loading labels from: {CSV_PATH}")
try:
    df = pd.read_csv(CSV_PATH)
except FileNotFoundError:
    print("❌ Error: CSV file not found at that path. Please check the file name.")
    exit()

Loading labels from: /content/Image_data.csv


In [5]:
print(f"\nLoading labels from: {CSV_PATH}")
df = pd.read_csv(CSV_PATH)

image_data = []
labels = []

print("Processing images...")
found_count = 0

for index, row in df.iterrows():
    # --- THE FIX IS HERE ---
    # Manually split by '\' to handle Windows paths on Linux
    raw_path = row['image_path']
    filename = raw_path.split('\\')[-1]

    # Clean up any potential whitespace
    filename = filename.strip()

    # Construct the full path for Colab
    full_path = os.path.join(IMAGE_FOLDER, filename)

    # Check and Load
    if os.path.exists(full_path):
        try:
            img = cv2.imread(full_path)
            if img is not None:
                img = cv2.resize(img, IMG_SIZE)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                flat_img = img.flatten() / 255.0

                image_data.append(flat_img)
                labels.append(row['target'])
                found_count += 1
        except Exception:
            pass

    # Progress
    if index > 0 and index % 2000 == 0:
        print(f"Checked {index} rows... Found {found_count} images so far.")


Loading labels from: /content/Image_data.csv
Processing images...
Checked 2000 rows... Found 2001 images so far.
Checked 4000 rows... Found 4001 images so far.
Checked 6000 rows... Found 6001 images so far.
Checked 8000 rows... Found 8001 images so far.
Checked 10000 rows... Found 10001 images so far.


In [6]:
# 3. Train Model
if found_count > 0:
    print(f"\n✅ Success! Loaded {len(image_data)} images.")

    # Convert to Numpy arrays
    X = np.array(image_data)
    y = np.array(labels)

    # Split Data (80% Train, 20% Test)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train SVM
    print("Training SVM Model... (This will take a few minutes)")
    svm_model = SVC(kernel='poly', degree=3)
    svm_model.fit(X_train, y_train)

    # Evaluate
    print("\n--- Evaluation Results ---")
    y_pred = svm_model.predict(X_test)
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(classification_report(y_test, y_pred))

else:
    print("\n❌ Error: Still found 0 images.")
    print(f"Double check this path: {IMAGE_FOLDER}")
    print("Verify an image exists there by running: !ls \"/content/machine_learning_project-/Project/Dataset/images\" | head")


✅ Success! Loaded 10480 images.
Training SVM Model... (This will take a few minutes)

--- Evaluation Results ---
Accuracy: 0.6861
              precision    recall  f1-score   support

           0       0.48      0.55      0.51       632
           1       0.79      0.74      0.77      1464

    accuracy                           0.69      2096
   macro avg       0.64      0.65      0.64      2096
weighted avg       0.70      0.69      0.69      2096

